#  Optical Coherence Tomography (OCT) and Chest X-Ray Images
 ## Project Overview: 
 This dataset contains thousands of validated OCT and Chest X-Ray images  described and analyzed in "Identifying Medical Diagnoses and Treatable Diseases by Image-Based Deep Learning". The images are split into a training set and a testing set of independent patients. Images are labeled as (disease)-(randomized patient ID)-(image number by this patient) and split into 4 directories: CNV, DME, DRUSEN, and NORMAL.

## Business Understating
The dataset has been split into two main parts: a training set and a testing set. These sets consist of images from different patients, and they are used to train and evaluate deep learning algorithms for medical diagnosis.

Each image is labeled with information indicating the type of disease, a randomized patient ID, and a unique image number for that patient. This labeling system helps in organizing and identifying specific images within the dataset.

## Problem Statement:
